In [31]:
import seaborn as sns
df=sns.load_dataset('tips')

In [32]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [34]:
df['day'].value_counts()


day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

In [35]:
df['smoker'].value_counts()

smoker
No     151
Yes     93
Name: count, dtype: int64

In [36]:
df['time'].value_counts()

time
Dinner    176
Lunch      68
Name: count, dtype: int64

In [37]:
df['sex'].value_counts()

sex
Male      157
Female     87
Name: count, dtype: int64

In [38]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [39]:
X=df[['tip', 'sex', 'smoker', 'day', 'time', 'size']]
y=df['total_bill']

In [40]:
##train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

In [41]:
X_train.head()

,tip,sex,smoker,day,time,size
58,1.76,Male,Yes,Sat,Dinner,2
1,1.66,Male,No,Sun,Dinner,3
2,3.50,Male,No,Sun,Dinner,3
68,2.01,Male,No,Sat,Dinner,2
184,3.00,Male,Yes,Sun,Dinner,2


In [42]:
##feature encoding by one hot encoding and label encoding

In [43]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
le1= LabelEncoder()
le2= LabelEncoder()
le3= LabelEncoder()



In [44]:
X_train['sex']=le1.fit_transform(X_train['sex'])
X_train['smoker']=le2.fit_transform(X_train['smoker'])
X_train['time']=le3.fit_transform(X_train['time'])

In [45]:
X_train.head()

,tip,sex,smoker,day,time,size
58,1.76,1,1,Sat,0,2
1,1.66,1,0,Sun,0,3
2,3.50,1,0,Sun,0,3
68,2.01,1,0,Sat,0,2
184,3.00,1,1,Sun,0,2


In [46]:
X_test['sex']=le1.transform(X_test['sex'])
X_test['smoker']=le2.transform(X_test['smoker'])
X_test['time']=le3.transform(X_test['time'])

In [47]:
X_test.head()     

,tip,sex,smoker,day,time,size
162,2.00,0,0,Sun,0,3
60,3.21,1,1,Sat,0,2
61,2.00,1,1,Sat,0,2
63,3.76,1,1,Sat,0,4
69,2.09,1,1,Sat,0,2


In [48]:
##one hot encoding for day column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [49]:
ct = ColumnTransformer(
    [('onehot', OneHotEncoder(drop='first'), [3])],
    remainder='passthrough'
)

In [50]:
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

X_train=ct.fit_transform(X_train)

In [51]:
X_test=ct.transform(X_test)

In [52]:
## svr-> support vector regression
from sklearn.svm import SVR
svr= SVR()

In [53]:
svr.fit(X_train, y_train)

SVR()

In [54]:
y_pred=svr.predict(X_test)

In [55]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_test, y_pred) 
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2 Score: {r2}')

MAE: 4.1486423210190235
MSE: 45.692136151683385
RMSE: 6.759595857126621
R2 Score: 0.46028114561159283


In [57]:
## hyperparameter tuning
from sklearn.model_selection import GridSearchCV
param_grid = { 'C': [0.1, 1, 10, 100,1000],'gamma': [0.001, 0.01, 0.1,0.0001],'kernel': ['linear', 'poly', 'rbf', 'sigmoid'] }


In [58]:
grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 80 candidates, totalling 400 fits


GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [0.001, 0.01, 0.1, 0.0001],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             verbose=2)

In [59]:
grid_search.best_params_

{'C': 100, 'gamma': 0.001, 'kernel': 'linear'}

In [60]:
grid_predictions = grid_search.predict(X_test)

In [61]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_test, y_pred) 
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2 Score: {r2}')

MAE: 4.1486423210190235
MSE: 45.692136151683385
RMSE: 6.759595857126621
R2 Score: 0.46028114561159283
